Imports necessários

In [1]:
import requests
from io import BytesIO
import zipfile
import pandas as pd
from sqlalchemy import create_engine

Download e extração

In [ ]:
# URL do arquivo compactado
url_do_arquivo_zip = 'https://portaldatransparencia.gov.br/download-de-dados/viagens/2023'

# Baixando o arquivo compactado
resposta = requests.get(url_do_arquivo_zip)

# Verificando se a solicitação foi bem-sucedida
if resposta.status_code == 200:
    # Lendo o conteúdo do arquivo compactado
    conteudo_zip = BytesIO(resposta.content)

    # Extraindo o conteúdo do arquivo compactado
    with zipfile.ZipFile(conteudo_zip, 'r') as zip_ref:
        # Local da extracao
        zip_ref.extractall('/csv_files')
    print("Conteúdo extraído com sucesso!")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {resposta.status_code}")

Leitura e transformacao dos arquivos .csv em DataFrame

In [73]:
arquivo_viagem = 'csv_files/2023_Viagem.csv'
arquivo_passagem = 'csv_files/2023_Passagem.csv'
arquivo_trecho = 'csv_files/2023_Trecho.csv'
arquivo_pagamento = 'csv_files/2023_Pagamento.csv'

# Carregando os dados do arquivo CSV em um DataFrame do pandas
#df_viagem = pd.read_csv(arquivo_viagem, sep=';', encoding="latin-1")

df_passagem = pd.read_csv(arquivo_passagem, sep=';', encoding="latin-1")

#df_trecho = pd.read_csv(arquivo_trecho, sep=';', encoding="latin-1")

#df_pagamento = pd.read_csv(arquivo_pagamento, sep=';', encoding="latin-1")

df_passagem.shape

(336077, 19)

Tratamento dos dados do dataframe viagem

Tratamento dos dados do dataframe passagens

In [74]:
colunas_numeric_passagem = ["Valor da passagem","Taxa de serviço"]
for coluna in colunas_numeric_passagem:
    df_passagem[coluna] = df_passagem[coluna].str.replace(',', '.')
    df_passagem[coluna] = df_passagem[coluna].apply(pd.to_numeric, errors='coerce')

for index, row in df_passagem.iterrows():
    if row[9] != 'Sem informação':

        row[15] = row[15]/2
        row[16] = row[16]/2

        df_passagem.loc[index] = row

        nova_linha = row.copy()

        nova_linha = [nova_linha[0], None, nova_linha[2], nova_linha[9], nova_linha[10], nova_linha[11], nova_linha[12], nova_linha[13], nova_linha[14], None,None,None,None,None,None, nova_linha[15], nova_linha[16], nova_linha[17], nova_linha[18]]

        df_passagem.loc[len(df_passagem)] = nova_linha

        '''tempo de execucao = 90m 55.4s'''

In [75]:
df_passagem

,Identificador do processo de viagem,Número da Proposta (PCDP),Meio de transporte,País - Origem ida,UF - Origem ida,Cidade - Origem ida,País - Destino ida,UF - Destino ida,Cidade - Destino ida,País - Origem volta,UF - Origem volta,Cidade - Origem volta,Pais - Destino volta,UF - Destino volta,Cidade - Destino volta,Valor da passagem,Taxa de serviço,Data da emissão/compra,Hora da emissão/compra
0,18288418,000007/23-1C,Aéreo,Brasil,Distrito Federal,Brasília,Estados Unidos da América,NaN,Nova York,Estados Unidos da América,NaN,Nova York,Brasil,Distrito Federal,Brasília,3717.065,0.00,04/11/2022,11:04
1,18302983,000001/23,Aéreo,Brasil,São Paulo,São Paulo,Suécia,NaN,Gotemburgo,Suécia,NaN,Gotemburgo,Brasil,São Paulo,São Paulo,4951.660,2499.89,15/09/2022,13:39
2,18306758,000002/23,Aéreo,Brasil,São Paulo,São Paulo,Suécia,NaN,Gotemburgo,Suécia,NaN,Gotemburgo,Brasil,São Paulo,São Paulo,4951.660,2499.89,15/09/2022,13:42
3,18306785,000004/23,Aéreo,Brasil,São Paulo,São Paulo,Suécia,NaN,Gotemburgo,Suécia,NaN,Gotemburgo,Brasil,São Paulo,São Paulo,4951.660,2499.89,15/09/2022,13:51
4,18306786,000003/23,Aéreo,Brasil,São Paulo,São Paulo,Suécia,NaN,Gotemburgo,Suécia,NaN,Gotemburgo,Brasil,São Paulo,São Paulo,4951.660,2499.89,15/09/2022,13:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363504,19627535,None,Aéreo,Brasil,Rio de Janeiro,Rio de Janeiro,Brasil,Distrito Federal,Brasília,None,None,None,None,None,None,2791.800,0.00,11/12/2023,11:53
363505,19628153,None,Aéreo,Brasil,Rio de Janeiro,Rio de Janeiro,Brasil,Distrito Federal,Brasília,None,None,None,None,None,None,1743.000,0.00,11/12/2023,13:15
363506,19629422,None,Aéreo,Brasil,Rio de Janeiro,Rio de Janeiro,Brasil,Distrito Federal,Brasília,None,None,None,None,None,None,2791.800,0.00,11/12/2023,11:21
363507,19634424,None,Aéreo,Brasil,Distrito Federal,Brasília,Brasil,Pernambuco,Recife,None,None,None,None,None,None,2117.240,0.00,11/12/2023,23:40
